In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
input = !ls /kaggle/input
input = '/kaggle/input/{}/'.format(input[0])
output = '/kaggle/working/'

print(input, output)

In [ ]:
## 基础工具
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import scipy
import math

np.random.seed(2021)

In [ ]:
def float_check(x):
    '''
    检测浮点数的小数位数
    9    103589 （保留）
    1     33551 （=0的需要处理，其它的酌情）
    8     10636 （保留）
    7      1047 （保留）
    6       135 （保留）
    4        76 （保留）
    3        56 （保留）
    5        51 （保留）
    2        22 （保留）
    0         1  drop
    '''
    if '.' in str(x):
        return(len(str(x).split('.')[-1]))
    else :
        return 0
    
def float_last(x):
    if '.' in str(x):
        return(str(x)[-1])
    else :
        return False
    
def sample_Interval(col, start, end, closed):
    '''
    去除首尾异常值，利用分组
    直接操作DataFrame对象
    start, end 是单个值
    '''
    z = 0.0000000001
    global df_train
    interval_index = pd.IntervalIndex.from_arrays([start], [end+z], closed=closed)
    x = pd.cut(df_train[col], interval_index, right=False)
    print(col,'drop：',x.isnull().sum())
    df_train = df_train[x.notnull()]
    

def make_box(col, box_index):
    '''
    col, start, end, closed 含义参考pd.IntervalIndex.from_arrays()
    做分组用
    type(n): int IntervalIndex
    n=5
    n=pd.IntervalIndex.from_arrays([start], [end+z], closed=closed)
    right:This argument is ignored when bins is an IntervalIndex.
    '''
    global df_train, box
    
    # 切割数据
    base = pd.IntervalIndex.from_arrays([1], [2], closed='left')
    if type(box_index)==type([]):
        n = box_index[0]
        right = True if box_index[1]=='right' else False
        cut = pd.cut(df_train[col], n, right=right)
    elif type(box_index)==type(base):
        interval_index = pd.IntervalIndex.from_arrays(box_index[0], box_index[1], closed=box_index[2])
        cut = pd.cut(df_train[col], interval_index)
    else:
        print("box_index 的类型错误")
        print(box_index)
        return 1
    
    Customer = df_train['SeriousDlqin2yrs'].groupby(cut).count()
    Bad = df_train['SeriousDlqin2yrs'].groupby(cut).sum()
    Good = Customer-Bad
    box_col = pd.merge(pd.DataFrame(Customer),pd.DataFrame(Bad) ,left_index=True,right_index=True)
    box_col.rename(columns={'SeriousDlqin2yrs_x':'CustomerNumber','SeriousDlqin2yrs_y':'Bad'},inplace=True)
    box_col.insert(0, "Feature", col)
    box_col.insert(len(box_col.columns), "Good", Customer-Bad)
    box_col.insert(len(box_col.columns), "BadRatio", Bad/Customer)
    box_col.index.name = 'Id'
    return box_col
    
def correlation_analysis():
    global box_col, features
    fig,subs=plt.subplots(math.ceil(len(features)/2), 2, figsize=(20,23))

    for i,col in enumerate(features):
        box_col = box[box['Feature']==col]
        sub_a = i//2
        sub_b = i%2
        # 图A
        labels = box_col.index.tolist()
        Good_data = box_col['Good']
        Bad_data = box_col['Bad']
        BadRatio_data = box_col['BadRatio']*box_col['CustomerNumber']
        # x轴是由[1,2,3,4,5] 组成
        x=np.arange(len(labels))
        width = 0.35
        #fig,ax  = plt.subplots()
        # 先画Good
        bar1 = subs[sub_a][sub_b].bar(x-width/2, Good_data, width, label='Good')
        for rect in bar1:
            height = rect.get_height()
            #annotate函数是用来对图像进行标注，
            #参数xy表示标注位置
            # xytext表示文本位置（在xy的位置进行多少偏移）
            subs[sub_a][sub_b].annotate('{}'.format(height),
                        xy=(rect.get_x()+rect.get_width()/2,height),
                        xytext=(0,3),
                        textcoords='offset points',
                        ha = 'center',
                        va = 'bottom',fontsize=8)
        # 再画Bad
        bar2 = subs[sub_a][sub_b].bar(x+width/2, Bad_data, width, label='Bad')
        # 再画折线
        subs[sub_a][sub_b].plot(BadRatio_data.to_list(), color='red', label='BadRatio')
        for rect in bar2:
            height = rect.get_height()
            #annotate函数是用来对图像进行标注，
            #参数xy表示标注位置
            # xytext表示文本位置（在xy的位置进行多少偏移）
            subs[sub_a][sub_b].annotate('{}'.format(height),
                        xy=(rect.get_x()+rect.get_width()/2,height),
                        xytext=(0,3),
                        textcoords='offset points',
                        ha = 'center',
                        va = 'bottom', fontsize=8)
        # 设置图像的一些参数
        subs[sub_a][sub_b].set_ylabel('Customer Number')
        subs[sub_a][sub_b].set_title('Customer Number by {}'.format(col))
        subs[sub_a][sub_b].set_xticks(x)
        subs[sub_a][sub_b].set_xticklabels(labels, rotation=15, fontsize=10)
        subs[sub_a][sub_b].legend()


    #tight_layout()，作用是自动调整子图参数，使之填充整个图像区域。
    fig.tight_layout()
    fig.show()

In [ ]:
# Step1： 加载数据
cs_training = pd.read_csv(input+'cs-training.csv')
cs_training = cs_training.iloc[:, 1:]
df_train = cs_training.copy()

In [ ]:
features = [*df_train.columns]
features.remove('SeriousDlqin2yrs')
describe = df_train.describe()

In [ ]:

# Step2：去除重复值并恢复索引
df_train.drop_duplicates(inplace=True)

# Step3：发现 “MonthlyIncome”和“NumberOfDependents” 包含大量缺失值
# 使用均值填补“NumberOfDependents”的缺失值
df_train["NumberOfDependents"].fillna(int(df_train["NumberOfDependents"].mean()),inplace=True)
# 使用均值填补 “MonthlyIncome” 的缺失值
df_train['MonthlyIncome'].fillna(df_train['MonthlyIncome'].mean(),inplace=True)

# 查看缺失值比例，预期不能有缺失值
display(df_train.isna().sum()/df_train.shape[0])

# Step4：处理异常值
# 描述性统计
display(df_train.describe([.01,.1,.25,.5,.75,.9,.99]).T)

# '''  异常值处理规则
# RevolvingUtilizationOfUnsecuredLines，[min, max+0.0000000001)
# age，[min, max+0.0000000001)
# NumberOfTime30-59DaysPastDueNotWorse， [min,80)
# NumberOfTime60-89DaysPastDueNotWorse，  [min,80)
# NumberOfTimes90DaysLate，  [min,80)
# NumberRealEstateLoansOrLines,  [min, max+0.0000000001)
# DebtRatio，应该是比率，23%的大于1,很多数据包含整数，最大值逆天了，先自动分组，然后处理异常值 [min, max+0.0000000001)
# MonthlyIncome， [min, max+0.0000000001)
# NumberOfOpenCreditLinesAndLoans，[min, max+0.0000000001)
# NumberOfDependents, [min, max+0.0000000001)
# '''
# 另一个可选的方案是将异常值分箱
# print(df_train.shape)
# col = 'RevolvingUtilizationOfUnsecuredLines'
# sample_Interval(col, describe[col]['min'], describe[col]['max'], 'left')
# col = 'NumberOfTime30-59DaysPastDueNotWorse'
# sample_Interval('NumberOfTime30-59DaysPastDueNotWorse', describe[col]['min'], 80, 'left')
# col = 'NumberOfTime60-89DaysPastDueNotWorse'
# sample_Interval('NumberOfTime60-89DaysPastDueNotWorse', describe[col]['min'], 80, 'left')
# col = 'NumberOfTimes90DaysLate'
# sample_Interval('NumberOfTimes90DaysLate', describe[col]['min'], 80, 'left')
# print(df_train.shape)


# Step5：columns转置成中文，由于不能解决kaggle画图显示中文的问题
# columns_translate ={'SeriousDlqin2yrs':'客户分类',
#         'RevolvingUtilizationOfUnsecuredLines':'可用额度比值',
#         'age':'年龄',
#         'NumberOfTime30-59DaysPastDueNotWorse':'逾期30-59天笔数',
#         'DebtRatio':'负债率',
#         'MonthlyIncome':'月收入',
#         'NumberOfOpenCreditLinesAndLoans':'信贷数量',
#         'NumberOfTimes90DaysLate':'逾期90天笔数',
#         'NumberRealEstateLoansOrLines':'固定资产贷款量',
#         'NumberOfTime60-89DaysPastDueNotWorse':'逾期60-89天笔数',
#         'NumberOfDependents':'家属数量'}
# column_bak = dict(map(reversed, columns_translate.items()))
# df_train.rename(columns=column_bak, inplace=True)
# df_train.head()    #修改英文字段名为中文字段名



#-----------------------------


In [ ]:
# Step6：相关性分析
box_index = {
    'RevolvingUtilizationOfUnsecuredLines': [4, 'left'],
    'age': [4,'left'],
    # 'age': pd.IntervalIndex.from_arrays([min], [max+z]], closed='left')
    # 'age': pd.IntervalIndex.from_arrays([21,31,41,51,61], [30,40,50,60,float('inf')]], closed='left')
    'NumberOfTime30-59DaysPastDueNotWorse': [4,'left'],
    'NumberOfTime60-89DaysPastDueNotWorse':[4,'left'],
    'NumberOfTimes90DaysLate': [4,'left'],
    'DebtRatio': [4,'left'],
    'MonthlyIncome': [4,'left'],
    'NumberOfOpenCreditLinesAndLoans': [4,'left'],
    'NumberRealEstateLoansOrLines': [4,'left'],
    'NumberOfDependents': [4,'left']
}
box = pd.DataFrame()
for col in features:
    box=box.append(make_box(col, box_index[col]))

# display(box)
# 等分的效果并不怎么样
correlation_analysis()

# 多变量分析
corr = df_train.corr()
cmap = sns.diverging_palette(200,20,sep=20,as_cmap=True)
f,ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corr, annot=True, cmap=cmap, annot_kws={'size':10}, linewidths=.5, fmt= '.3f',ax=ax)
plt.show()


In [ ]:
# Step7：样本不均衡问题
X = df_train.drop('SeriousDlqin2yrs',axis=1)
y = df_train['SeriousDlqin2yrs']
sns.countplot(x='SeriousDlqin2yrs', data=df_train)
plt.show()
# 使用SMOTE方法进行过抽样处理
from imblearn.over_sampling import SMOTE # 过抽样处理库SMOTE
model_smote = SMOTE() # 建立SMOTE模型对象
X,y = model_smote.fit_resample(X,y) # 输入数据并作过抽样处理
smote_resampled = pd.concat([X, y],axis=1) # 按列合并数据框
groupby_data_smote = smote_resampled.groupby('SeriousDlqin2yrs').count() # 对label做分类汇总
groupby_data_smote # 打印输出经过SMOTE处理后的数据集样本分类分布
sns.countplot(x='SeriousDlqin2yrs',data=smote_resampled)
plt.show()

# 该方法导致AUC低于0.8

In [ ]:
# 切分数据集
from sklearn.model_selection import train_test_split
x = df_train.drop('SeriousDlqin2yrs',axis=1)
y = df_train['SeriousDlqin2yrs']

x_train,x_vali,y_train,y_vali = train_test_split(x,y,test_size=0.3,random_state=2021)

train_data = pd.concat([y_train,x_train],axis=1)
train_data.index = range(train_data.shape[0])
train_data.columns = df_train.columns

test_data = pd.concat([y_vali,x_vali],axis=1)
test_data.index = range(test_data.shape[0])
test_data.columns = df_train.columns

# Step8： 找到合适的分箱方法

In [ ]:
'''
有监督方法：基于sklearn决策树的最优分箱与IV值计算
'''
from sklearn.tree import DecisionTreeClassifier

def optimal_binning_boundary(x: pd.Series, y: pd.Series, nan: float = -999.) -> list:
    '''
        利用决策树获得最优分箱的边界值列表
    '''
    boundary = []  # 待return的分箱边界值列表
    
    x = x.fillna(nan).values  # 填充缺失值
    y = y.values
    
    clf = DecisionTreeClassifier(criterion='entropy',    #“信息熵”最小化准则划分
                                 max_leaf_nodes=6,       # 最大叶子节点数
                                 min_samples_leaf=0.05)  # 叶子节点样本数量最小占比
    
    clf.fit(x.reshape(-1, 1), y)  # 训练决策树
    
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    threshold = clf.tree_.threshold
    
    for i in range(n_nodes):
        if children_left[i] != children_right[i]:  # 获得决策树节点上的划分边界值
            boundary.append(threshold[i])
    
    boundary.sort()
    
    z = 0.0000000001
    min_x = x.min()
    max_x = x.max() + z  # +0.1是为了考虑后续groupby操作时，能包含特征最大值的样本
    boundary = [min_x] + boundary + [max_x]
    
    return boundary

In [ ]:
# 测试optimal_binning_boundary函数：
optimal_binning_boundary(x=train_data['RevolvingUtilizationOfUnsecuredLines'],
                         y=train_data['SeriousDlqin2yrs'])

In [ ]:
# 获得某个变量各个分箱的WOE、IV值函数的实现：
def feature_woe_iv(feature: str, x: pd.Series, y: pd.Series, nan: float = -999.) -> pd.DataFrame:
    '''
        计算变量各个分箱的WOE、IV值，返回一个DataFrame
    '''
    x = x.fillna(nan)
    boundary = optimal_binning_boundary(x, y, nan)        # 获得最优分箱边界值列表
    df = pd.concat([x, y], axis=1)                        # 合并x、y为一个DataFrame，方便后续计算
    df.columns = ['x', 'y']                               # 特征变量、目标变量字段的重命名
    df['bins'] = pd.cut(x=x, bins=boundary, right=False)  # 获得每个x值所在的分箱区间
    
    grouped = df.groupby('bins')['y']                     # 统计各分箱区间的好、坏、总客户数量
    result_df = grouped.agg([('good',  lambda y: (y == 0).sum()), 
                             ('bad',   lambda y: (y == 1).sum()),
                             ('total', 'count')])

    result_df['good_pct'] = result_df['good'] / result_df['good'].sum()       # 好客户占比
    result_df['bad_pct'] = result_df['bad'] / result_df['bad'].sum()          # 坏客户占比
    result_df['total_pct'] = result_df['total'] / result_df['total'].sum()    # 总客户占比

    result_df['bad_rate'] = result_df['bad'] / result_df['total']             # 坏比率
    
    result_df['woe'] = np.log(result_df['good_pct'] / result_df['bad_pct'])              # WOE
    result_df['iv'] = (result_df['good_pct'] - result_df['bad_pct']) * result_df['woe']  # IV
    
    print(f"'{feature}'该变量IV = {result_df['iv'].sum()}")
    
    return result_df,boundary

In [ ]:
# 决策树分箱
box = {}
boundary = {}
for col in features:
    box[col],boundary[col] = feature_woe_iv(col, x=train_data[col], y=train_data['SeriousDlqin2yrs'])
    display(box[col].style.bar(color=['#d65f5f', '#5fba7d'], align='mid', subset=['bad_rate','woe']))

RevolvingUtilizationOfUnsecuredLines: 是理想中的分箱，保持了woe的单调性

age: 保持了woe的单调性

NumberOfTime30-59DaysPastDueNotWorse： 分箱还可以

DebtRatio、MonthlyIncome：暂且归为U型变量，不做处理

NumberOfOpenCreditLinesAndLoans： 不作处理

NumberOfTimes90DaysLate: 分箱还可以

NumberRealEstateLoansOrLines： 不作处理

NumberOfTime60-89DaysPastDueNotWorse： 分箱还可以

NumberOfDependents：合并inf出现的box

In [ ]:
# 如果需要分箱，调整 boundary
# DebtRatio，应该是比率，23%的大于1,很多数据包含整数，最大值逆天了，先自动分组，然后处理异常值 [min, max+0.0000000001)
col = 'DebtRatio'
print(boundary[col])
box[col].style.bar(color=['#d65f5f', '#5fba7d'], align='mid', subset=['bad_rate','woe'])
# 处理结果还可以

In [ ]:
def feature_woe_iv_new(feature: str, boundary, x: pd.Series, y: pd.Series, nan: float = -999.) -> pd.DataFrame:
    '''
        计算变量各个分箱的WOE、IV值，返回一个DataFrame
    '''
    x = x.fillna(nan)
    #boundary = optimal_binning_boundary(x, y, nan)        # 获得最优分箱边界值列表
    df = pd.concat([x, y], axis=1)                        # 合并x、y为一个DataFrame，方便后续计算
    df.columns = ['x', 'y']                               # 特征变量、目标变量字段的重命名
    df['bins'] = pd.cut(x=x, bins=boundary, right=False)  # 获得每个x值所在的分箱区间
    
    grouped = df.groupby('bins')['y']                     # 统计各分箱区间的好、坏、总客户数量
    result_df = grouped.agg([('good',  lambda y: (y == 0).sum()), 
                             ('bad',   lambda y: (y == 1).sum()),
                             ('total', 'count')])

    result_df['good_pct'] = result_df['good'] / result_df['good'].sum()       # 好客户占比
    result_df['bad_pct'] = result_df['bad'] / result_df['bad'].sum()          # 坏客户占比
    result_df['total_pct'] = result_df['total'] / result_df['total'].sum()    # 总客户占比

    result_df['bad_rate'] = result_df['bad'] / result_df['total']             # 坏比率
    
    result_df['woe'] = np.log(result_df['good_pct'] / result_df['bad_pct'])              # WOE
    result_df['iv'] = (result_df['good_pct'] - result_df['bad_pct']) * result_df['woe']  # IV
    
    print(f"'{feature}'该变量IV = {result_df['iv'].sum()}")
    
    return result_df

# 重新分箱
# box_new = {}
# for col in cs_training.columns:
#     box_new[col] = feature_woe_iv_new(col, boundary[col], x=cs_training[col], y=cs_training['SeriousDlqin2yrs'])
#     #display(box_new[col].style.bar(color="skyblue", subset=['bad_rate']))

一般认为，IV值小于0.03的特征几乎不带有有效信息，对模型没有贡献，可以删除，这组特征中最低值为’NumberOfDependents’为0.028，
差别不大，暂且保留查看效果

In [ ]:
# Step9:
# 将原数据分箱后，按箱的结果把WOE结构用map函数映射到数据中
train_woe = pd.DataFrame()
for col in boundary:
    if col == 'SeriousDlqin2yrs': continue
    train_woe[col] = pd.cut(train_data[col],boundary[col], right=False).map(box[col]['woe'])
    
#将标签补充到数据中
train_woe["SeriousDlqin2yrs"] = train_data["SeriousDlqin2yrs"]
train_woe

In [ ]:
train_woe.isnull().sum()

In [ ]:
display(train_data[train_woe['MonthlyIncome'].isnull()])
display(train_woe[train_woe['MonthlyIncome'].isnull()])
train_woe = train_woe.dropna()
train_woe.isnull().sum()

In [ ]:
#将原数据分箱后，按箱的结果把WOE结构用map函数映射到数据中
test_woe = pd.DataFrame()
for col in boundary:
    if col == 'SeriousDlqin2yrs': continue
    test_woe[col] = pd.cut(test_data[col], boundary[col], right=False).map(box[col]['woe'])
    
#将标签补充到数据中
test_woe["SeriousDlqin2yrs"] = test_data["SeriousDlqin2yrs"]
test_woe.isnull().sum()

In [ ]:
display(test_data[test_woe['RevolvingUtilizationOfUnsecuredLines'].isnull()])
display(test_woe[test_woe['RevolvingUtilizationOfUnsecuredLines'].isnull()])

display(test_data[test_woe['DebtRatio'].isnull()])
display(test_woe[test_woe['DebtRatio'].isnull()])

display(test_data[test_woe['NumberOfOpenCreditLinesAndLoans'].isnull()])
display(test_woe[test_woe['NumberOfOpenCreditLinesAndLoans'].isnull()])

display(test_data[test_woe['NumberRealEstateLoansOrLines'].isnull()])
display(test_woe[test_woe['NumberRealEstateLoansOrLines'].isnull()])

display(test_data[test_woe['NumberOfDependents'].isnull()])
display(test_woe[test_woe['NumberOfDependents'].isnull()])
test_woe = test_woe.dropna()
test_woe.isnull().sum()

In [ ]:
# 处理测试集
X_test = test_woe.iloc[:,:-1]
y_test = test_woe.iloc[:,-1]
# 建模
X_train = train_woe.iloc[:,:-1]
y_train = train_woe.iloc[:,-1]

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression as LR
lr = LR(random_state=2021)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
#y_pred2 = lr.predict_proba(X_test)
print('Accuracy: ', accuracy_score(y_pred,y_test))
# 尽量让AUC>=0.8，提升AUC的方法：
# 1） IV筛选可以降低0.02
# 2） 调整分箱
# 3） 构造特征
print('AUC:' , roc_auc_score(y_pred,y_test))
print('F1:', f1_score(y_pred, y_test))

AUC > 0.9 ，说明效果还不错

In [ ]:
# sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)
# 混淆矩阵
from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test, y_pred)
sns.heatmap(C2,annot=True,fmt='d')

绘制ROC曲线，ROC=0.94，曲线越往左上凸，True Positive 就越高，对应的False Positive越低。

In [ ]:
import scikitplot as skplt
vali_proba_df = pd.DataFrame(lr.predict_proba(X_test))
skplt.metrics.plot_roc(y_test, vali_proba_df,
                      plot_micro=False,figsize=(6,6),
                      plot_macro=False)

# 4 制作评分卡

求出A、B和base_score

将所有特征的评分卡内容全部一次性写往一个本地文件ScoreData.csv：

In [ ]:
A = 650
B = 72.13

features
boundary
# 生成评分卡

def generate_scorecard(model_coef,  features, B):
    global box
    coef = model_coef[0]
    # print(coef)
    
    cols = ['Variable', 'Binning', 'Score']
    for i in range(len(features)):
        f = features[i]
        lst = []
        # 筛选该f的 WOE规则
        for index, row in box[f].iterrows():
            lst.append(int(round(-B*coef[i]*row['woe'])))
        box[f]['score'] = lst
        
# 评分卡计算 score
generate_scorecard(lr.coef_,  features, B)
for index in box:
    display(box[index])

In [ ]:
#  这里记录了kai'ke'ba以一种计算方法
def str_to_int(x):
    if x=='-inf':
        return -999999
    if x=='inf':
        return 999999
    return float(x)

# 将value映射到bin
def map_value_to_bin(feature_value, feature_to_bin):
    for index, row in feature_to_bin.iterrows():
        bins =  str(row.name)
        binnings = bins[1:-1].split(',')
        in_range = True
        if feature_value <= str_to_int(binnings[0]):
            in_range = False
        if feature_value > str_to_int(binnings[1]):
            in_range = False
        if in_range:
            return row.name
    return None

def map_to_score(df, score_card):
    # score_card = box_new
    # 得到评分卡规则中的字段
    score_columns = list(score_card.keys())
    score_columns.remove('SeriousDlqin2yrs')
    # 累加 col 的 score
    for col in score_columns:
        # 得到关于col的规则
        feature_to_bin = score_card[col]
        # df样本中的col
        feature_value = df[col]
        #　将ｃｏｌ数值　映射到　Ｂｉｎｎｉｎｇ
        selected_bin = map_value_to_bin(feature_value, feature_to_bin)
        # 累加score
        selected_score = feature_to_bin[selected_bin]['score'].iloc[0]
        # 累加到整体的score
        score += selected_score
    return score


# 按照评分卡规则 score card 计算df中的分数
def cal_score_with_card(df, score_card, A):
    df['score'] = df.apply(map_to_score, args=(score_card, ),axis=1)
    df['score'] = df['score'] + A
    return df

In [ ]:
# 按照评分卡规则 score card 计算df中的分数
def cal_score_with_card(df, score_card, A):
    df_score = df.copy()
    
    # score_card = box_new
    # 得到评分卡规则中的字段
    columns = list(score_card.keys())
    
    score_columns = []
    # 累加 col 的 score
    for col in columns:
        df_score['score_{}'.format(col)] = df_score[col].map(score_card[col]['score'])
        score_columns.append('score_{}'.format(col))
        
    df_score['score'] = df_score[score_columns].sum(axis=1)
    df_score['score'] = df_score['score'] + A
    return df_score

In [ ]:
# 随机选择Good的5个
good_sample = df_train[df_train['SeriousDlqin2yrs']==0].sample(5,random_state=2022)
good_sample = good_sample[features]
good_sample

# 计算分数
good_sample_score = cal_score_with_card(good_sample, box, 650)

good_sample_score

In [ ]:
# 随机选择Good的5个
bad_sample = df_train[df_train['SeriousDlqin2yrs']==1].sample(5,random_state=2022)
bad_sample = good_sample[features]
bad_sample

# 计算分数
bad_sample_score = cal_score_with_card(good_sample, box, 650)

bad_sample_score

In [ ]:
# 计算分数
cs_training_score = cal_score_with_card(df_train, box, 650)
cs_training_score.to_csv('cs_training_score.csv', header=True)
cs_training_score

In [ ]:
cs_test = pd.read_csv(input+'cs-test.csv')
cs_test = cs_test.iloc[:, 1:]

# 计算分数
cs_test_score = cal_score_with_card(cs_test, box, 650)
cs_test_score.to_csv('cs_test_score.csv', header=True)
cs_test_score

In [ ]:
sampleEntry = pd.read_csv(input+'sampleEntry.csv', index_col='Id')

sampleEntry

In [ ]:
cs_test = pd.read_csv(input+'cs-test.csv', index_col=0)
cs_test = cs_test.drop(['SeriousDlqin2yrs'], axis=1)

display(cs_test.isna().sum()/cs_test.shape[0])

#使用均值填补“NumberOfDependents”的缺失值
cs_test["NumberOfDependents"].fillna(int(cs_test["NumberOfDependents"].mean()),inplace=True)
# 使用均值填补 “MonthlyIncome” 的缺失值
cs_test['MonthlyIncome'].fillna(cs_test['MonthlyIncome'].mean(),inplace=True)

display(cs_test.isna().sum()/cs_test.shape[0])
    
#将原数据分箱后，按箱的结果把WOE结构用map函数映射到数据中
cs_test_woe = pd.DataFrame()
for col in boundary:
    if col == 'SeriousDlqin2yrs': continue
    cs_test_woe[col] = pd.cut(cs_test[col], boundary[col], right=False).map(box[col]['woe'])

display(cs_test_woe.isnull().sum())

In [ ]:
cs_test_woe['MonthlyIncome'].fillna(cs_test_woe['MonthlyIncome'].value_counts().index[0], inplace=True)
cs_test_woe['NumberOfOpenCreditLinesAndLoans'].fillna(cs_test_woe['NumberOfOpenCreditLinesAndLoans'].value_counts().index[0], inplace=True)
cs_test_woe['NumberOfDependents'].fillna(cs_test_woe['NumberOfDependents'].value_counts().index[0], inplace=True)
display(cs_test_woe.isnull().sum())

In [ ]:

y_pred2 = lr.predict_proba(cs_test_woe)


predict = pd.DataFrame(index=cs_test.index)
predict['Probability'] = y_pred2[:, 1]
predict.index.name = 'Id'
predict

In [ ]:
predict.to_csv('predict.csv', header=True)